In [1]:
from kaggle_secrets import UserSecretsClient


In [3]:
import more_itertools
import torch
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from huggingface_hub import login
from transformers import pipeline
from sklearn.metrics import roc_auc_score


2025-10-17 04:53:48.071984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760676828.106588     197 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760676828.116161     197 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")   # use the label you created

#hf_token =os.getenv("HF_TOKEN")
login(hf_token#, add_to_git_credential=True
 )

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto


tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-mamba-7b-instruct")
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-mamba-7b-instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-68f1cf07-1d006f214ae55fad59d74599;a5013ff4-f4b2-4964-9711-6eaa90ef7426)

Entry Not Found for url: https://huggingface.co/api/models/tiiuae/falcon-mamba-7b-instruct/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [ ]:



# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"      # automatically uses GPU/CPU
)

In [ ]:
from transformers import pipeline

# Emotion-based
emotion_pipe = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion", return_all_scores=True)

# Domain-specific
domain_pipe = pipeline("text-classification", model="unitary/toxic-bert", return_all_scores=True)

text = "jack off now she is for free http://codimenu.com/ac/16php"

print("Emotion Analysis:", emotion_pipe(text))
print("Domain Analysis:", domain_pipe(text))


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
GEMMA_PATH = kagglehub.model_download("google/gemma-3/transformers/gemma-3-1b-it")
processor = AutoTokenizer.from_pretrained(GEMMA_PATH)

# Determine if CUDA (GPU) is available
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(GEMMA_PATH).to(device)
print(model)

In [9]:
test_data = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules-enforcement/test.csv')

In [15]:
import pandas as pd
from transformers import pipeline

#train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/test.csv')
def get_sentiment(sent_pipe, input: pd.Series):
    return "".join([str(x['label'])+ " : "+ str(x['score']) for x in sent_pipe(input['subreddit'])[0]])

# Emotion-based
emotion_pipe = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion", return_all_scores=True)

# Domain-specific
domain_pipe = pipeline("text-classification", model="unitary/toxic-bert", return_all_scores=True)
domain_pipedata=[]
emotion_pipedata=[]
for batch in more_itertools.batched(test_data.iterrows(), 4):
    emotion_pipedata+= [get_sentiment(emotion_pipe, x) for _, x in batch]
    domain_pipedata+= [get_sentiment(domain_pipe, x) for _, x in batch]
test_data['emotion_pipe']=emotion_pipedata
test_data['domain_pipe']=domain_pipedata

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-68f1d0b9-6b20c7070d69471045b13823;3587455e-1c71-43f7-96d7-7d6c272a2c46)

Entry Not Found for url: https://huggingface.co/api/models/bhadresh-savani/bert-base-uncased-emotion/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [10]:
def prompt(input: pd.Series):
    return """<start_of_turn>user
You are a really experienced moderator for the subreddit /r/%s. Your job
is to determine if the following reported comments violates the rule:
%s

%s
Decision:
True

%s
Decision:
False

%s
Decision:
False

%s
Decision:
True

%s
<end_of_turn>
<start_of_turn>model\n""" % (
    input['subreddit'],
    input['rule'],
    "\n".join(["| " + x for x in input['positive_example_1'].split('\n')]),
    "\n".join(["| " + x for x in input['negative_example_1'].split('\n')]),
    "\n".join(["| " + x for x in input['negative_example_2'].split('\n')]),
    "\n".join(["| " + x for x in input['positive_example_2'].split('\n')]),
    "\n".join(["| " + x for x in input['body'].split('\n')])    
)
             

In [ ]:
token_ids = [processor.get_vocab()[word] for word in ['True', 'False']]
if any(token_id == processor.get_vocab()['<unk>'] for token_id in token_ids):
      raise ValueError('One of the target classes is not in the vocabulary.')

In [11]:
responses = []
for batch in more_itertools.batched(test_data.iterrows(), 4):
    prompts = [prompt(x) for _, x in batch]
    pre = processor(text=prompts, return_tensors="pt", padding=True, truncation=True,
                    max_length=512).to(device)
    with torch.no_grad():
      outputs = model(**pre)
    logits = outputs.logits[:, -1, token_ids]  
    probabilities = torch.softmax(logits, dim=-1)
    responses.extend(probabilities[:, 0].tolist())  

NameError: name 'processor' is not defined

In [ ]:
my_submission = pd.DataFrame({
    'row_id': test_data['row_id'],
    'rule_violation': responses
})

In [ ]:
my_submission.to_csv('submission.csv', index=False)